Mini-Project 2: Performing a spectral analysis of a GRB 
 

In this project you will be performing a full spectral analysis of a GRB from start to finish, starting with a basic data product from the Fermi Gamma-Ray Burst Monitor (GBM) and finishing with the best fit spectral model parameters and their errors. This process will closely follow what is done by the Fermi team themselves. The process will have three major steps,

1. Calculate the background-subtracted counts in each of the 8 energy bins over the desired time interval
2. Setup the functions needed to calculate the expected counts in each energy bin for a spectral model (power-law spectra)
3. Find the best fit spectral model parameters and their errors using chi2 fitting 
 
You will be walked through the exact process below with several part to each major step. This process will string together skills and methods used in previous homeworks and tutorials. 

 

As usual, remember to comment your code where appropriate, include docstrings for any functions that you write, label all of your plot axes, and include a legend and labels on each plot that has more than one curve plotted. You may work on this assignment by yourself or with a partner. If you work with a partner, both partners should turn in a notebook and make sure to indicate at the top who they worked with. 

Part 1 - Setting up the data
Just as we did in tutorial 16, we will start some light curve data that has the observed counts in each time interval for 8 different energy bins. 

You can download the data from: [HERE](https://psu.instructure.com/courses/2367528/files/176346222/download?download_frd=1)

A. Using astropy's Table object read in the data. There are two tables in this FITS file that we want. The 3rd HDU (index 2) is the light curve data table and the 2nd HDU (index 1) is the measured energy bin table. The measured energy bins in this table correspond to the min and max measured energies for the 8 bins of counts in the light curve data table. 
- Read in both of these tables
- Print their column names
- Show the energy bin table (execute a cell containing just the table variable name or print it)

In [ ]:
#import all required libraries
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy.io import fits

from scipy import stats

ImportError: cannot import name 'Table' from 'astropy' (c:\Users\Insan\AppData\Local\Programs\Python\Python311\Lib\site-packages\astropy\__init__.py)

In [ ]:
filemane = 'https://psu.instructure.com/courses/2367528/files/176346222/download?download_frd=1'
